In [1]:
using GynC
using GynC: sample, hzobj, Config, Lausanne

thin=1
K=1000
M=3
niter=10;

10

In [2]:
@time ss=[sample(Config(Lausanne(m), thin=thin), K) for m in 1:M];

 90.139822 seconds (1.26 G allocations: 19.779 GB, 6.88% gc time)


In [3]:
@time pi1=WeightedChain(ss);

  8.958810 seconds (13.37 M allocations: 428.521 MB, 6.91% gc time)


In [4]:
pin=deepcopy(pi1);
@time for i in 1:niter
    emiteration!(pin)
end

  0.038738 seconds (8.58 k allocations: 389.996 KB)


In [5]:
L

LoadError: LoadError: UndefVarError: L not defined
while loading In[5], in expression starting on line 1